#### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [ ]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup
from datetime import datetime

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re
# Activating the chrome browser

driver = webdriver.Chrome(r'C:\Users\udayakiran\Desktop\chromedriver.exe')
driver.get("https://www.amazon.in/")
search_box = driver.find_element_by_id("twotabsearchtextbox")
print('Website Opened and waiting for input : ', datetime.now().time())
search_box.send_keys(input())
search_button = driver.find_element_by_id("nav-search-submit-button")
search_button.click()                                            # clearing search_bar  

#### Q2. 2.	In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [ ]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

In [ ]:
Brand,Rating,No_of_Ratings,Price,Product_Name,Return_or_Exchange,Expected_Delivery,Availability,otherdetails,url_line=[],[],[],[],[],[],[],[],[],[]
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    try:
        Brand.append(driver.find_element_by_xpath('//a[@id="bylineInfo"]').text)      # Extracting Brand from xpath
    except NoSuchElementException:
        Brand.append('-')
    try:
        Rating.append(driver.find_element_by_xpath('//span[@id="acrPopover"]').get_attribute("title"))  # Extracting Ratings from xpath
    except NoSuchElementException:
        Rating.append('-')
    try:
        No_of_Ratings.append(driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span').text)     # Extracting no. of Ratings from xpath
    except NoSuchElementException:
        No_of_Ratings.append('-')
    try:
        Price.append(driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text) # Extracting Price from xpath
    except NoSuchElementException:
        try:
            Price.append(driver.find_element_by_xpath('//span[@id="priceblock_saleprice"]').text)
        except NoSuchElementException:
            Price.append('-')    
    try:
        Product_Name.append(driver.find_element_by_xpath('//h1[@id="title"]/span').text)      # Extracting Name from xpath
    except NoSuchElementException:
        Product_Name.append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        Return_or_Exchange.append(driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a').text)
    except NoSuchElementException:
        Return_or_Exchange.append('-')
    try:
        Expected_Delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text)         # Extracting Expected Delivery from xpath
    except NoSuchElementException:
        Expected_Delivery.append('-')    
    try:
        Availability.append(driver.find_element_by_xpath('//div[@id="availability"]/span').text)                # Extracting Availability from xpath
    except NoSuchElementException:
        Availability.append('-')
    try:                                                                                    # Extracting Other Details from xpath
        otherdetails.append(driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]').text)
    except NoSuchElementException:
        otherdetails.append('-')    
    url_line.append(url) 
Item_lst = pd.DataFrame({'Brand_Name' : pd.Series(Brand),
                         'Product Name' :pd.Series(Product_Name),
                         'Item_Price' : pd.Series(Price),
                         'Rating' : pd.Series(Rating),
                         'No_of_Ratings':pd.Series(No_of_Ratings),
                         'Return/Exchange' :pd.Series(Return_or_Exchange),
                         'Expected Delivery' : pd.Series(Expected_Delivery),
                         'Availability': pd.Series(Availability),
                         'otherdetails': pd.Series(otherdetails)
                        })
compression_opts = dict(method='zip', archive_name='Amazon_report.csv')
Item_lst.to_csv('Amazon_report.zip', index=False, compression=compression_opts)

#### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [ ]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
driver = webdriver.Chrome(r'C:\Users\udayakiran\Desktop\chromedriver.exe')
# Opening images.google.com
driver.get('https://images.google.com/')

#### Fruits scraping

In [ ]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()     # Clicking the search button

print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("F:/image_scrap/"+str(i)+".jpg", "wb")
    file.write(response.content)

#### Cars Scraping

In [ ]:
driver = webdriver.Chrome(r'C:\Users\udayakiran\Desktop\chromedriver.exe')
driver.get('https://images.google.com/')
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("Cars")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()     # Clicking the search button

print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("F:/Car_Image/"+str(i)+".jpg", "wb")
    file.write(response.content)

In [ ]:
driver = webdriver.Chrome(r'C:\Users\udayakiran\Desktop\chromedriver.exe')
driver.get('https://images.google.com/')
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("Machine Learning")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()     # Clicking the search button

print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("F:/ML_Images/"+str(i)+".jpg", "wb")
    file.write(response.content)

#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
# Asking for user input
prod = input(" Enter the name of the mobile phone you want to search : ")
driver.get('https://www.flipkart.com/')
time.sleep(3)
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath
search_bar.clear()               # Clearing the search bar
search_bar.send_keys(prod)       # Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [ ]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [ ]:
len(flip_urls)

In [ ]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [ ]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')
    
                                                               

In [ ]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))

In [ ]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

#### Q5. 5.	Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

#### Q6.	Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [ ]:
driver.get('https://trak.in/')

In [ ]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [ ]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [ ]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

In [ ]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")

#### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [ ]:
driver.get('https://www.digit.in/')

In [ ]:
#clickng on top 10 option 
top_10=driver.find_element_by_xpath("//div[@class='menu']/ul[1]/li[3]/a[1]")
top_10.click()

#clicking on laptops option
laptops=driver.find_element_by_xpath("//div[@class='categoty_list']/button[2]")
laptops.click()

#best gaming laptops link
best_gaming=driver.find_element_by_xpath("//div[@id='laptops']/div[3]/a[1]")
driver.get(best_gaming.get_attribute('href'))

#intialising lists
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

#names
for i in driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3"):
    name.append(i.text)
    
#os
for i in driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div"):
    OS.append(i.text)
    
#display
for i in driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div"):
    display.append(i.text)
    
#processor
for i in driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div"):
    processor.append(i.text)

#memory
memories = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")
memories_spec = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]") #values of specifiations 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')#append no details as value for memory is missing in some of the laptops
            RAM.append('No details available')#append no details as value for memory is missing in some of the laptops

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")#getting the url of full specs links
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:#iterating throug every laptops full specs' page
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:#exception handling for no price details
        price.append("No details available")
        
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

In [ ]:
df.to_csv("Gaming laptops_digit.csv")